In [1]:
import pandas as pd
import numpy as np
import xmltodict
import requests
import json
from collections import OrderedDict

In [2]:
SQL_CONN = 'sqlite:////Users/alex/Downloads/data'

In [3]:
# activities              exercises_musclegroups  workout_exercises     
#                         exercises_muscles       workouts              
# equipments              muscle_groups         
# exercises               muscles

In [4]:
muscle_groups = pd.io.sql.read_sql_table('muscle_groups', SQL_CONN)

In [5]:
muscles = pd.io.sql.read_sql_table('muscles', SQL_CONN)

In [6]:
exercises = pd.io.sql.read_sql_table('exercises', SQL_CONN)

In [7]:
exercises_musclegroups = pd.io.sql.read_sql_table('exercises_musclegroups', SQL_CONN)

In [8]:
exercises_muscles = pd.io.sql.read_sql_table('exercises_muscles', SQL_CONN)

In [9]:
# pd.io.sql.read_sql_table('equipments', SQL_CONN)

In [10]:
activities = pd.io.sql.read_sql_table('activities', SQL_CONN)

In [11]:
workout_exercises = pd.io.sql.read_sql_table('workout_exercises', SQL_CONN)

In [12]:
workouts = pd.io.sql.read_sql_table('workouts', SQL_CONN)

In [13]:
# muscle_groups_txt = """const MUSCLE_GROUPS = [
# %s
# ];

# export default MUSCLE_GROUPS;
# """ % ',\n'.join(["  {id: %d, name: '%s'}" % (x._1, x.name) for x in muscle_groups.itertuples()])

In [14]:
# with open('/Users/alex/git/aFitness2/constants/MuscleGroups.js', 'w') as fp:
#     print>>fp, muscle_groups_txt,

In [15]:
# muscles_txt ="""const MUSCLES = [
# %s
# ];

# export default MUSCLES;
# """ % ',\n'.join(['  {"id": %d, "name": "%s", "muscleGroup": %d}' % (x._1, x.name, x.musclegroup_id ) for x in muscles.itertuples()])

In [16]:
# with open('/Users/alex/git/aFitness2/constants/Muscles.js', 'w') as fp:
#     print>>fp, muscles_txt,

In [17]:
import json

In [18]:
exercises_clean = exercises.drop(labels=['guid'], axis=1)

In [19]:
exercises_muscles_clean = exercises_muscles.drop(labels=['_id'], axis=1).set_index('exercise_id')
exercises_muscles_clean = exercises_muscles_clean[exercises_muscles_clean.muscle_id != -1]

In [20]:
def get_exercises(x):
    try:
        entry = exercises_muscles_clean.ix[x]
    except KeyError:
        return []
    if isinstance(entry, pd.DataFrame):
        return entry.to_dict('records')
    else:
        return [entry.to_dict()]

exercises_clean['muscles'] = exercises_clean._id.apply(get_exercises)

In [21]:
exercises_musclegroups_clean = exercises_musclegroups.drop(labels=['_id'], axis=1).set_index('exercise_id')
exercises_musclegroups_clean = exercises_musclegroups_clean[exercises_musclegroups_clean.musclegroup_id != -1]

In [22]:
# def get_mg(x):
#     try:
#         return [{'musclegroup_id': int(exercises_musclegroups_clean.ix[x])}]
#     except KeyError:
#         return []
# exercises_clean['muscle_groups'] = exercises_clean._id.apply(get_mg)

In [23]:
def do_require(image):
    if image:
        return 'xxxxx%syyyyy' % image

In [24]:
def clear_nans(x):
    return dict((k,v) for k,v in x.iteritems() if (v == v) and v is not None)

In [25]:
def _get_sub(i):
    new_item = dict(**i)
    name = new_item.pop('@name')
    return name, parse_item(new_item)

def parse_item(item):
    if isinstance(item, (OrderedDict, dict)):
        if len(item) == 1 and '@name' in item:
            return {item['@name']: {}}
        else:
            return dict((k, parse_item(v)) for k, v in item.iteritems())
    elif isinstance(item, list):
        return dict([_get_sub(i) for i in item])
    else:
        return item

In [26]:
xml_content = requests.get('https://raw.githubusercontent.com/cancan101/aFitness/master/app/src/main/res/xml/exercises.xml').content

xml_data = xmltodict.parse(xml_content)
xml_exercises = xml_data['Exercises']['Exercise']

parsed = parse_item(xml_exercises)

xml_load = pd.DataFrame.from_dict(parsed, orient='index').reset_index()

In [27]:
merged = exercises_clean.merge(xml_load, left_on='name', right_on='index', how='left')

In [28]:
images = pd.DataFrame({'image': exercises['image'][~exercises.image.isnull()].unique()})

In [29]:
images_df = images.set_index('image', drop=False)
images_txt = json.dumps(
    images_df.image.apply(do_require).to_dict(), indent=2)

In [30]:
images_txt_final = """const IMAGES = %s;

export default IMAGES;
""" % '\n'.join([a.rstrip() for a in images_txt.splitlines()]).replace('"xxxxx', "require('../img/").replace('yyyyy"', "_t.jpg')")

In [31]:
with open('/Users/alex/git/aFitness2/constants/Images.js', 'w') as fp:
    print>>fp, images_txt_final,

In [32]:
merged_both = merged[~pd.isnull(merged['index'])]

In [33]:
def extract_muscle(x):
    if x != x or x is None:
        return []
    else:
        return x['Muscle'].keys()

secondary_new = merged.SecondaryMuscles.apply(extract_muscle)
primary_new = merged.MajorMuscles.apply(extract_muscle)

In [34]:
primaries_now = merged.muscles.apply(lambda y: [x for x in y if x['is_primary']==1])
secondary_now = merged.muscles.apply(lambda y: [x for x in y if x['is_primary']!=1])

In [35]:
assert 0 == (merged_both.image_x != merged_both.image_y).sum()

In [36]:
assert 0 == ((primaries_now.apply(len) != primary_new.apply(len)) & (~pd.isnull(merged['index']))).sum()

In [37]:
primaries_now_name = primaries_now.apply(lambda x: set(muscles.set_index('_id').ix[y['muscle_id'], 'name'] for y in x))
primary_new[
    (primaries_now_name.apply(lambda y: set(x.lower() for x in y)) != primary_new.apply(lambda x: set(y.lower() for y in x)))
     & (~pd.isnull(merged['index']))
]

Series([], Name: MajorMuscles, dtype: object)

In [38]:
scondary_now_name = secondary_now.apply(lambda x: set(muscles.set_index('_id').ix[y['muscle_id'], 'name'] for y in x))
secondary_new[
    (scondary_now_name.apply(lambda y: set(x.lower() for x in y)) != secondary_new.apply(lambda x: set(y.lower() for y in x)))
     & (~pd.isnull(merged['index']))
]

Series([], Name: SecondaryMuscles, dtype: object)

In [39]:
assert 0 == (secondary_now.apply(len) != secondary_now.apply(len)).sum()

In [40]:
muscles_raw = xmltodict.parse(
    requests.get(
        'https://raw.githubusercontent.com/cancan101/aFitness/9917d0b5ebadd2d0e2e386c2426f74feae4b6ee1/app/src/main/assets/muscles.xml').content)

In [41]:
muscles_d = parse_item(muscles_raw)["MuscleGroups"]["MusclesGroup"]
muscles_d['Cardio'] = {"Muscle": {}}

with open('/Users/alex/git/aFitness2/data/muscles.json', 'w') as fp:
    print>>fp, '\n'.join([s.rstrip() for s in json.dumps(muscles_d, indent=2).splitlines()],)

In [42]:
merged.SecondaryMuscles = merged.SecondaryMuscles.fillna(scondary_now_name.apply(
        lambda x: np.nan if x != x or not x else {u'Muscle': {y: {} for y in x}}))
merged.MajorMuscles = merged.MajorMuscles.fillna(primaries_now_name.apply(lambda x: {u'Muscle': {y: {} for y in x}}))

exercises_clean_save = pd.concat([
        merged[['description', 'difficulty', 'type', 'target1', 'target2', 'target3', 'SecondaryMuscles', 'MajorMuscles']],
        exercises_clean], axis=1)

exercises_txt = json.dumps(
    [clear_nans(x) for x in exercises_clean_save.rename(columns={'_id': 'id'}).drop(labels=['muscles'], axis=1).to_dict(orient='records')],
    indent=2, sort_keys=True)

with open('/Users/alex/git/aFitness2/data/exercises.json', 'w') as fp:
    print>>fp, '\n'.join([a.rstrip() for a in exercises_txt.splitlines()]),